In [1]:
text = open('../data/raw_2014.txt', 'r').read()

In [2]:
def q_to_b(q_str):
    """
    功能：非中文文字的全角转半角
    输入：一个字符串
    输出：半角字符串
    
    """
    b_str = ""
    for uchar in q_str:
        inside_code = ord(uchar)
        if inside_code == 12288:  
            inside_code = 32
        elif 65374 >= inside_code >= 65281:  
            inside_code -= 65248
        b_str += chr(inside_code)
    return b_str


In [3]:
text = q_to_b(text)

In [5]:
sentences = text.split('\n')

In [6]:
print('num_seqence:', len(sentences))

num_seqence: 286269


In [7]:
sentences[0]

'人民网/nz 1月1日/t 讯/ng 据/p 《/w [纽约/nsf 时报/n]/nz 》/w 报道/v ,/w 美国/nsf 华尔街/nsf 股市/n 在/p 2013年/t 的/ude1 最后/f 一天/mq 继续/v 上涨/vn ,/w 和/cc [全球/n 股市/n]/nz 一样/uyy ,/w 都/d 以/p [最高/a 纪录/n]/nz 或/c 接近/v [最高/a 纪录/n]/nz 结束/v 本/rz 年/qt 的/ude1 交易/vn 。/w '

In [8]:
import re

def sent_sep(sent):
    """将句子和标注分割"""
    sent = sent.strip()
    
    # 处理 '[]'
    if '[' in sent:
        sent = sent.replace(' [',' ')
        sent = re.sub(r']/[a-z]+', '', sent)
    
    sent = sent.split(' ')
    
    # 处理空
    sent = [item for item in sent if '/' in item]
    
    sents, tags = [], []
    for item in sent:
        tmp = item.split('/')
        try:
            sents.append(tmp[0])
            tags.append(tmp[1])
        except:
            print(sent)
    return sents, tags


In [9]:
new_sentences, new_targets = [], []
for sent in sentences:
    sents,tags = sent_sep(sent)
    new_sentences.append(sents)
    new_targets.append(tags)

In [10]:
print(new_sentences[0])

['人民网', '1月1日', '讯', '据', '《', '纽约', '时报', '》', '报道', ',', '美国', '华尔街', '股市', '在', '2013年', '的', '最后', '一天', '继续', '上涨', ',', '和', '全球', '股市', '一样', ',', '都', '以', '最高', '纪录', '或', '接近', '最高', '纪录', '结束', '本', '年', '的', '交易', '。']


In [11]:
print(new_targets[0])

['nz', 't', 'ng', 'p', 'w', 'nsf', 'n', 'w', 'v', 'w', 'nsf', 'nsf', 'n', 'p', 't', 'ude1', 'f', 'mq', 'v', 'vn', 'w', 'cc', 'n', 'n', 'uyy', 'w', 'd', 'p', 'a', 'n', 'c', 'v', 'a', 'n', 'v', 'rz', 'qt', 'ude1', 'vn', 'w']


In [12]:
def tag_process(tags):
    """
    给 人物 地点 机构 时间 打上 PER，LOC，ORG，T 标签
    其他为 O
    采用BIO标记
    """
    new_tags = []
    for tag in tags:
        if tag == 't':
            new_tag = 'T'
        elif tag.startswith('nr'):
            new_tag = 'PER'
        elif tag.startswith('ns'):
            new_tag = 'LOC'
        elif tag.startswith('nt'):
            new_tag = 'ORG'
        else:
            new_tag  = 'O'
        new_tags.append(new_tag)
        
    tags = []
    for index, tag in enumerate(new_tags):
        if tag in ['PER', 'LOC','T','ORG']:
            if index==0:
                new_tag = 'B_'+tag
            else:
                if new_tags[index-1]=='O':
                    new_tag = 'B_'+tag
                else:
                    new_tag = 'I_'+tag
        else:
            new_tag = tag
        tags.append(new_tag)
    return tags
    

In [13]:
targets = []
for tags in new_targets:
    tags = tag_process(tags)
    targets.append(tags)

In [14]:
print(targets[0])

['O', 'B_T', 'O', 'O', 'O', 'B_LOC', 'O', 'O', 'O', 'O', 'B_LOC', 'I_LOC', 'O', 'O', 'B_T', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [15]:
import pickle
with open('../outputs/source_word_2014_cropus.pkl','wb') as fw:
    fw.write(pickle.dumps(new_sentences))
with open('../outputs/target_word_2014_cropus.pkl','wb') as fw:
    fw.write(pickle.dumps(targets))

In [16]:
#　合并词和标签, 至此，基于词的处理就结束了
res = []
for word, tag in zip(new_sentences[0], targets[0]):
    res.append(word+'/'+tag)
print(' '.join(res))

人民网/O 1月1日/B_T 讯/O 据/O 《/O 纽约/B_LOC 时报/O 》/O 报道/O ,/O 美国/B_LOC 华尔街/I_LOC 股市/O 在/O 2013年/B_T 的/O 最后/O 一天/O 继续/O 上涨/O ,/O 和/O 全球/O 股市/O 一样/O ,/O 都/O 以/O 最高/O 纪录/O 或/O 接近/O 最高/O 纪录/O 结束/O 本/O 年/O 的/O 交易/O 。/O


In [17]:
print(new_sentences[0])

['人民网', '1月1日', '讯', '据', '《', '纽约', '时报', '》', '报道', ',', '美国', '华尔街', '股市', '在', '2013年', '的', '最后', '一天', '继续', '上涨', ',', '和', '全球', '股市', '一样', ',', '都', '以', '最高', '纪录', '或', '接近', '最高', '纪录', '结束', '本', '年', '的', '交易', '。']


### 因为后边的模型是基于字符的，所以这里还要做进一步处理

In [18]:
def sep(sent,labels):
    new_sent = []
    new_labels = []
    for s, label in zip(sent,labels):
        s = list(s)
#         print(s)
        n = len(s)
        if n==1:
            new_labels.append(label)
            new_sent.append(s[0])
        else:
            for i in range(n):
                if i > 0:
                    label = label.replace('B','I')
                new_labels.append(label)
                new_sent.append(s[i])
#     print('new_sent', new_sent)
#     print('new_labels',new_labels)
    return new_sent,new_labels
        
def init_seq(sentences, targets):
    """将词分割成字符，字符的标签同词的标签"""
    new_sentences = []
    new_targets = []
    for sent, labels in zip(sentences,targets):
        new_sent, new_labels = sep(sent,labels)
        new_sentences.append(new_sent)
        new_targets.append(new_labels)
    return new_sentences, new_targets    
    

In [19]:
new_sent, new_labels = sep(new_sentences[0],targets[0])
res=[]
for sent, lable in zip(new_sent, new_labels):
    res.append(sent+'/'+lable)
print(' '.join(res))

人/O 民/O 网/O 1/B_T 月/I_T 1/I_T 日/I_T 讯/O 据/O 《/O 纽/B_LOC 约/I_LOC 时/O 报/O 》/O 报/O 道/O ,/O 美/B_LOC 国/I_LOC 华/I_LOC 尔/I_LOC 街/I_LOC 股/O 市/O 在/O 2/B_T 0/I_T 1/I_T 3/I_T 年/I_T 的/O 最/O 后/O 一/O 天/O 继/O 续/O 上/O 涨/O ,/O 和/O 全/O 球/O 股/O 市/O 一/O 样/O ,/O 都/O 以/O 最/O 高/O 纪/O 录/O 或/O 接/O 近/O 最/O 高/O 纪/O 录/O 结/O 束/O 本/O 年/O 的/O 交/O 易/O 。/O


In [20]:
new_sentences, new_targets = init_seq(new_sentences,targets)

In [21]:
print(new_sentences[0])

['人', '民', '网', '1', '月', '1', '日', '讯', '据', '《', '纽', '约', '时', '报', '》', '报', '道', ',', '美', '国', '华', '尔', '街', '股', '市', '在', '2', '0', '1', '3', '年', '的', '最', '后', '一', '天', '继', '续', '上', '涨', ',', '和', '全', '球', '股', '市', '一', '样', ',', '都', '以', '最', '高', '纪', '录', '或', '接', '近', '最', '高', '纪', '录', '结', '束', '本', '年', '的', '交', '易', '。']


In [22]:
print(new_targets[0])

['O', 'O', 'O', 'B_T', 'I_T', 'I_T', 'I_T', 'O', 'O', 'O', 'B_LOC', 'I_LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B_LOC', 'I_LOC', 'I_LOC', 'I_LOC', 'I_LOC', 'O', 'O', 'O', 'B_T', 'I_T', 'I_T', 'I_T', 'I_T', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [23]:
res = []
for word, tag in zip(new_sentences[0], new_targets[0]):
    res.append(word+'/'+tag)
print(' '.join(res))

人/O 民/O 网/O 1/B_T 月/I_T 1/I_T 日/I_T 讯/O 据/O 《/O 纽/B_LOC 约/I_LOC 时/O 报/O 》/O 报/O 道/O ,/O 美/B_LOC 国/I_LOC 华/I_LOC 尔/I_LOC 街/I_LOC 股/O 市/O 在/O 2/B_T 0/I_T 1/I_T 3/I_T 年/I_T 的/O 最/O 后/O 一/O 天/O 继/O 续/O 上/O 涨/O ,/O 和/O 全/O 球/O 股/O 市/O 一/O 样/O ,/O 都/O 以/O 最/O 高/O 纪/O 录/O 或/O 接/O 近/O 最/O 高/O 纪/O 录/O 结/O 束/O 本/O 年/O 的/O 交/O 易/O 。/O


In [24]:
# 保存
with open('../outputs/source_BIO_2014_cropus.txt','w') as fw:
    for sent in new_sentences:
        fw.write(" ".join(sent)+'\n')
with open('../outputs/target_BIO_2014_cropus.txt','w') as fw:
    for sent in new_targets:
        fw.write(" ".join(sent)+'\n')
            